# Учим w2v на сказках

In [1]:
# Ссылка на выбранное вами произведение
# Я взял преступление и наказание (ненвижу Достоевского)
url = 'https://raw.githubusercontent.com/nevmenandr/word2vec-russian-novels/master/books_before/CrimeAndPunishment.txt'

In [7]:
import requests
resp = requests.get(url)
text = resp.text

In [9]:
text[:300]

'ПРЕСТУПЛЕНИЕ И НАКАЗАНИЕ\nРОМАН \nВ ШЕСТИ ЧАСТЯХ С ЭПИЛОГОМ\n\nЧАСТЬ ПЕРВАЯ\n\nI\n\nВ начале июля, в чрезвычайно жаркое время, под вечер, один молодой человек вышел из своей каморки, которую нанимал от жильцов в С - м переулке, на улицу и медленно, как бы в нерешимости, отправился к К - ну мосту.\nОн благопо'

In [10]:
len(text)

1076543

In [12]:
text = text.lower()
print(text[:300])

преступление и наказание
роман 
в шести частях с эпилогом

часть первая

i

в начале июля, в чрезвычайно жаркое время, под вечер, один молодой человек вышел из своей каморки, которую нанимал от жильцов в с - м переулке, на улицу и медленно, как бы в нерешимости, отправился к к - ну мосту.
он благопо


!pip install nltk

In [ ]:
import nltk 
nltk.download('punkt')

In [15]:
from nltk.tokenize import sent_tokenize
sentences = sent_tokenize(text)
sentences[100]

'- я вам проценты еще за месяц внесу; потерпите.'

In [16]:
len(sentences)

13702

In [20]:
from nltk.tokenize import word_tokenize
words0 = word_tokenize(sentences[0])
words0[:5]

['преступление', 'и', 'наказание', 'роман', 'в']

In [22]:
sent_words = [word_tokenize(sent) for sent in sentences]

In [24]:
words = [word  for sent in sent_words for word in sent]
len(words)

221280

In [26]:
from collections import Counter
cnt = Counter(words)

In [32]:
' ' in words

False

In [29]:
cnt.most_common()[:10]

[(',', 25337),
 ('и', 8451),
 ('.', 8222),
 ('-', 6065),
 ('не', 3756),
 ('в', 3727),
 ('!', 3265),
 ('что', 3196),
 ('он', 2849),
 ('на', 2402)]

In [25]:
len(set(words))

26116

Стоп слова в топку!

In [ ]:
nltk.download("stopwords")

In [34]:
from nltk.corpus import stopwords
stopwords_ru = stopwords.words('russian') 
stopwords_ru[:10]

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со']

In [40]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [41]:
sent_words = [[word for word in sent 
               if word not in stopwords_ru and len(word) > 1 
               and word not in string.punctuation] 
              for sent in sent_words]

In [42]:
sent_words[0]

['преступление',
 'наказание',
 'роман',
 'шести',
 'частях',
 'эпилогом',
 'часть',
 'первая',
 'начале',
 'июля',
 'чрезвычайно',
 'жаркое',
 'время',
 'вечер',
 'молодой',
 'человек',
 'вышел',
 'своей',
 'каморки',
 'которую',
 'нанимал',
 'жильцов',
 'переулке',
 'улицу',
 'медленно',
 'нерешимости',
 'отправился',
 'мосту']

In [48]:
from pymorphy2 import MorphAnalyzer
m = MorphAnalyzer()
m.normal_forms('стали')

['стать', 'сталь']

In [53]:
m.normal_forms('бутявковедами')

['бутявковед']

In [52]:
m.parse('стали')

[Parse(word='стали', tag=OpencorporaTag('VERB,perf,intr plur,past,indc'), normal_form='стать', score=0.984662, methods_stack=((<DictionaryAnalyzer>, 'стали', 904, 4),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,gent'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 1),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,datv'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 2),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,loct'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 5),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,nomn'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 6),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,accs'), normal_form='сталь', score=0.003067, methods_stack=((<DictionaryAnalyzer>, 'стали', 13, 9),))]

In [61]:
text = "Филипп пошёл в Авиньон и пленил пап!"
tokens = word_tokenize(text)

" ".join(m.normal_forms(token)[0] for token in tokens)

'филипп пойти в авиньон и пленить папа !'

In [62]:
sent_words = [[m.normal_forms(word)[0]  for word in sent 
               if word not in stopwords_ru and len(word) > 1 
               and word not in string.punctuation] 
              for sent in sent_words]

## Обучение w2v 